In [2]:
from __future__ import print_function, division, unicode_literals
import six
import os
from os.path import join
import json
from codecs import open
from collections import defaultdict
from operator import itemgetter
import nltk
import numpy as np
from nltk.corpus import stopwords
import re
import codecs

import spacy

from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("english")

In [3]:
import spacy.en

In [7]:
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english')) + ""

In [4]:
nlp = spacy.en.English()

In [24]:
def tokenize(text):
    return [stemmer.stem(w) for w in nltk.word_tokenize(text.lower()) if w not in stopwords]
#     return [stemmer.stem(w.text) for w in nlp(text) if not w.is_stop and not w.is_punct and not w.is_space]

In [25]:
import lucene

In [26]:
from org.apache.lucene.analysis.standard import StandardAnalyzer
from org.apache.lucene.document import Document, Field
from org.apache.lucene.index import IndexWriter, IndexWriterConfig, IndexReader
from org.apache.lucene.search import IndexSearcher
from org.apache.lucene.search import Sort, SortField
from org.apache.lucene.queryparser.classic import QueryParser
from org.apache.lucene.store import SimpleFSDirectory
from org.apache.lucene.util import Version

In [27]:
from java.io import File

In [28]:
DATA_DIR = join(os.environ['HOME'], 'data/allen-ai-challenge')
WIKI_DIR = join(DATA_DIR, 'wiki_dump')
CK12_DIR = join(DATA_DIR, 'ck12_dump')
TRAINING_SET = join(DATA_DIR, 'training_set.tsv')
VALIDATION_SET = join(DATA_DIR, 'validation_set.tsv')
TRAINING_SET_MERGED = join(DATA_DIR, 'training_set_merged.tsv')
INDEX_DIR = join(DATA_DIR, 'index-wiki-ck12')
# INDEX_DIR = join(DATA_DIR, 'index-ck12-stem')
INDEX_DIR = join(DATA_DIR, 'index-all-l_stem_summ')
SUBMISSION = join(DATA_DIR, 'submissions/lucene_wiki_ck12_16jan.tsv')

In [29]:
lucene.initVM()

In [30]:
analyzer = StandardAnalyzer(Version.LUCENE_4_10_1)

In [31]:
writerConfig = IndexWriterConfig(Version.LUCENE_4_10_1, StandardAnalyzer())
writer = IndexWriter(SimpleFSDirectory(File(INDEX_DIR)), writerConfig)

In [32]:
%%time
for i, fn_short in enumerate(os.listdir(CK12_DIR)):
    fn = join(CK12_DIR, fn_short)
    with open(fn, encoding='utf-8', errors='ignore') as f:
        ck12_article = json.load(f)
        content = ck12_article["science"] + " " + ck12_article["concept"] + " " + '. '.join([paragraph for subtitle, paragraph in ck12_article['contents'].items()])
        content = " ".join(tokenize(content))
        doc = Document()
        doc.add(Field("text", content, Field.Store.YES, Field.Index.ANALYZED))
        writer.addDocument(doc)

CPU times: user 25 s, sys: 585 ms, total: 25.6 s
Wall time: 26.1 s


In [1]:
# for i, fn_short in enumerate(os.listdir(WIKI_DIR)):
#     fn = join(WIKI_DIR, fn_short)
#     with open(fn, encoding='utf-8', errors='ignore') as f:
#         wiki_article = json.load(f)
#         a, b, content = wiki_article
#         print(content)
# #         content = " ".join(tokenize(content[0]))
#         if i > 2:
#             break

In [33]:
%%time
for i, fn_short in enumerate(os.listdir(WIKI_DIR)):
    fn = join(WIKI_DIR, fn_short)
    with open(fn, encoding='utf-8', errors='ignore') as f:
        wiki_article = json.load(f)
        _, summary, content = wiki_article
        content = summary + " " + content
        content = " ".join(tokenize(content))
        doc = Document()
        doc.add(Field("text", content, Field.Store.YES, Field.Index.ANALYZED))
        writer.addDocument(doc)

CPU times: user 6min 46s, sys: 6.97 s, total: 6min 53s
Wall time: 7min 11s


In [34]:
import telepot
def telegram_notify(msg):
    token = "178350795:AAFG7yae2SSt52GLek2bKS43oK7BaywWxRw"
    bot = telepot.Bot(token)
    b = bot.getMe()
    response = bot.getUpdates()
    bot.sendMessage(31747780, msg)

In [35]:
telegram_notify("lucene done!")

In [36]:
writer.numDocs()

13635

In [37]:
writer.close()

In [38]:
def iter_data(datafile, with_correct=True):
    with open(datafile, encoding='utf-8', errors='ignore') as f:
        next(f)
        for l in f:
            if with_correct:
                idd, q, correct, aa, ab, ac, ad = l.strip().split("\t")
            else:
                idd, q, aa, ab, ac, ad = l.strip().split("\t")
                correct = "no"
            q, aa, ab, ac, ad = [' '.join(tokenize(x)) for x in [q, aa, ab, ac, ad]]
            yield {"idd": idd, "q": q, "correct": correct, "aa": aa, "ab": ab, "ac": ac, "ad": ad}

In [39]:
from collections import defaultdict

In [44]:
%%time
res = defaultdict(list)
MAX = 10
docs_per_q = range(1, 20)

analyzer = StandardAnalyzer(Version.LUCENE_4_10_1)
reader = IndexReader.open(SimpleFSDirectory(File(INDEX_DIR)))
searcher = IndexSearcher(reader)

for row in iter_data(TRAINING_SET):
    queries = [row['aa'], row['ab'], row['ac'], row['ad']]
    queries = [row['q'] + ' ' + q  for q in queries]
    scores = defaultdict(list)
    for q in queries:
        query = QueryParser(Version.LUCENE_4_10_1, "text", analyzer).parse(re.sub("[^a-zA-Z0-9]"," ", q))
        #query = QueryParser(Version.LUCENE_30, "text", analyzer).parse(re.sub("[/^]", "\^", q))
        hits = searcher.search(query, MAX)
        doc_importance = [hit.score for hit in hits.scoreDocs]
        for n in docs_per_q:
            scores[n].append(sum(doc_importance[:n]))
      
    for n in docs_per_q:
        res[n].append(['A','B','C','D'][np.argmax(scores[n])] == row["correct"])

CPU times: user 17.3 s, sys: 1.78 s, total: 19.1 s
Wall time: 19.7 s


In [43]:
for x in sorted(res):
    print(x, np.mean(res[x]))

1 0.428
2 0.43
3 0.4328
4 0.4332
5 0.434
6 0.4384
7 0.4364
8 0.4336
9 0.4288


In [64]:
%%time
MAX = 100
docs_per_q = 4

analyzer = StandardAnalyzer(Version.LUCENE_4_10_1)
reader = IndexReader.open(SimpleFSDirectory(File(INDEX_DIR)))
searcher = IndexSearcher(reader)

with open(SUBMISSION, "w") as s:
    s.write("id,correctAnswer\n")
    for row in iter_data(VALIDATION_SET, False):
        queries = [row['aa'], row['ab'], row['ac'], row['ad']]
        queries = [row['q'] + ' ' + q  for q in queries]
        scores = []
        for q in queries:
            query = QueryParser(Version.LUCENE_4_10_1, "text", analyzer).parse(re.sub("[^a-zA-Z0-9]"," ", q))
            #query = QueryParser(Version.LUCENE_30, "text", analyzer).parse(re.sub("[/^]", "\^", q))
            hits = searcher.search(query, MAX)
            doc_importance = [hit.score for hit in hits.scoreDocs]
            scores.append(sum(doc_importance[:2]))
        guess = "ABCD"[np.argmax(scores)]
        s.write("%s,%s\n" % (row["idd"], guess))

CPU times: user 1min 4s, sys: 7.01 s, total: 1min 11s
Wall time: 1min 16s


Features
-----

In [71]:
FEATURES_LUCENE_TOP100_SCORES = join(DATA_DIR, 'features/lucene_top100.tsv')
FEATURES_LUCENE_ALL_SCORES = join(DATA_DIR, 'features/lucene_all.tsv')

In [72]:
%%time
MAX = 100
analyzer = StandardAnalyzer(Version.LUCENE_4_10_1)
reader = IndexReader.open(SimpleFSDirectory(File(INDEX_DIR)))
searcher = IndexSearcher(reader)

with open(FEATURES_LUCENE_TOP100_SCORES, "w") as fs:
    for row in iter_data(TRAINING_SET):
        queries = [row['aa'], row['ab'], row['ac'], row['ad']]
        queries = [row['q'] + ' ' + q  for q in queries]
        features = []
        for q in queries:
            query = QueryParser(Version.LUCENE_4_10_1, "text", analyzer).parse(re.sub("[^a-zA-Z0-9]"," ", q))
            hits = searcher.search(query, MAX)
            doc_importances = [str(hit.score) for hit in hits.scoreDocs]
            features.append(";".join(doc_importances))
        print(row["idd"], row["correct"], *features, file=fs, sep="\t")

CPU times: user 26.4 s, sys: 4.1 s, total: 30.5 s
Wall time: 32.1 s


In [98]:
sorter = Sort()
sorter.setSort(SortField("content", SortField.Type.STRING))

In [99]:
%%time
MAX = 10
analyzer = StandardAnalyzer(Version.LUCENE_4_10_1)
reader = IndexReader.open(SimpleFSDirectory(File(INDEX_DIR)))
searcher = IndexSearcher(reader)

with open(FEATURES_LUCENE_ALL_SCORES, "w") as fs:
    for row in iter_data(TRAINING_SET):
        queries = [row['aa'], row['ab'], row['ac'], row['ad']]
        queries = [row['q'] + ' ' + q  for q in queries]
        features = []
        for q in queries:
            query = QueryParser(Version.LUCENE_4_10_1, "text", analyzer).parse(re.sub("[^a-zA-Z0-9]"," ", q))      
            hits = searcher.search(query, MAX, sorter)
            doc_importances = [str(hit.score) for hit in hits.scoreDocs]
            features.append(";".join(doc_importances))
            print(doc_importances)
            break
        print(row["idd"], row["correct"], *features, file=fs, sep="\t")
        break

['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan']
CPU times: user 22.3 ms, sys: 3.8 ms, total: 26.1 ms
Wall time: 19.7 ms
